In [1]:
import config
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf;

train = pd.read_pickle(config.TRAIN)
test = pd.read_pickle(config.TEST)
tf.__version__

2022-11-24 05:10:19.348027: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 05:10:19.534765: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 05:10:19.534799: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-24 05:10:20.468547: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

'2.11.0'

## preprocessing with tensorflow

In [2]:
train["problem_statement"]

678D     find long function four onli note valu integ s...
915F     follow maximum connect simpl differ function p...
708B     input a11 four exist determin prove integ nega...
256B     adjac initi four space left squar integ coordi...
1535E    bufferedwrit input solv initi left cours integ...
                               ...                        
990C     follow name bracket concaten "()()()". oper ar...
1154C    input correspond stew long plan four sunday in...
339C     pan correspond order zero left integ littl spe...
909D     input order oper four left integ etc color lef...
582E     follow five correspond variabl oper four space...
Name: problem_statement, Length: 6272, dtype: object

In [3]:
# train data 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
MAX_SEQUENCE_LENGTH = 174 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train["problem_statement"])
X_train = tokenizer.texts_to_sequences(train["problem_statement"])
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
X_train.shape

(6272, 174)

In [4]:
def y_pick_tag(y, _tag):
    """ for training only on _tag"""
    y[[_tag]]=1
    for idx in y.index:
        if _tag not in y.loc[idx,"tags"]:
            y.loc[idx,_tag] = 0
    return y[_tag]

y_train = train[["tags"]]
y_train = y_pick_tag(y_train, "implementation")
y_train = np.array(y_train)
y_train.shape

(6272,)

In [5]:
# test data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test["problem_statement"])
X_test = tokenizer.texts_to_sequences(test["problem_statement"])
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

y_test = test[["tags"]]
y_test = y_pick_tag(y_test, "implementation")
y_test = np.array(y_test)
X_test.shape, y_test.shape

((1568, 174), (1568,))

## Baseline.
baseline. logistic regression on tag: implementation

In [6]:
# logistic regression with sklearn for baseline
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression(solver="sag",random_state=config.SEED)
log_clf.fit(X_train, y_train);
y_pred = log_clf.predict(X_test);

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
# 0.7136479591836735 

0.7136479591836735


/home/jhojin/CODE/codeforces_tag_predict/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## RNN
https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/blob/master/4.TEXT_CLASSIFICATION/4.1.6%20RNN%20Classification.ipynb

In [7]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6272, 174), (6272,), (1568, 174), (1568,))

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
tf.random.set_seed(config.SEED)

model_name = 'rnn_classifier_en'
BATCH_SIZE = 128
NUM_EPOCHS = 5
VALID_SPLIT = 0.1
MAX_LEN = X_train.shape[1]
VOCAB_SIZE = 200
WORD_EMBEDDING_DIM = 100
HIDDEN_STATE_DIM = 150
DENSE_FEATURE_DIM = 150
DATA_OUT_PATH = "./data/"

kargs = {'model_name': model_name,
        'vocab_size': VOCAB_SIZE,
        'embedding_dimension': 100,
        'dropout_rate': 0.2,
        'lstm_dimension': 150,
        'dense_dimension': 150,
        'output_dimension':1}

class RNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                     output_dim=kargs['embedding_dimension'])
        self.lstm_1_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'], return_sequences=True)
        self.lstm_2_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'])
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(units=kargs['dense_dimension'],
                           activation=tf.keras.activations.tanh)
        self.fc2 = layers.Dense(units=kargs['output_dimension'],
                           activation=tf.keras.activations.sigmoid)
    
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.lstm_1_layer(x)
        x = self.lstm_2_layer(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


In [9]:
model = RNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])


2022-11-24 05:10:25.325433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-24 05:10:25.325504: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-24 05:10:25.325542: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HJ-ThinkPadX1C8): /proc/driver/nvidia/version does not exist
2022-11-24 05:10:25.325891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=1)
checkpoint_path = DATA_OUT_PATH + model_name + '/weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)


In [11]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'rnn_classifier_en/embedding/embedding_lookup' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/jhojin/CODE/codeforces_tag_predict/.venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jhojin/CODE/codeforces_tag_predict/.venv/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/jhojin/CODE/codeforces_tag_predict/.venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/jhojin/CODE/codeforces_tag_predict/.venv/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loo